### Important points to note :
Hashlib contains SHA-256 which will be used to calculate **block_hash**

In [53]:
import hashlib
import os, sys
from typing import List, Optional

### Transaction

#### Attributes

**from_account** : Account from which money was sent.

**to_account** : Account to which money is credited.

**amount** : Amount transferred.

**incentive** : transactions often have some incentive to be added to the next block like some transaction fees.

In [54]:
class Transaction:
    # Constructor
    def __init__(self,from_account: str,to_account: str,amount: int,incentive: int):
        self.from_account = from_account
        self.to_account = to_account
        self.amount = amount
        self.incentive = incentive
    

#### Function to Calculate Hash

In [55]:
def calculate_hash(data: bytes)->str:
    ans =  hashlib.sha3_256(data).hexdigest()
    #print("Calculated hash value = " + ans)
    return ans

### Merkle Tree

#### Attributes

**hash_value** : if both children, hash(node) = hash(hash(l)+hash(r)). else, hash(node) = hash(left)

**left** : Pointer to left

**right** : Pointer to right

In [56]:
class MerkleNode:
    # For leaf nodes, left and right is None
    def __init__(self,hash_value: str,left: Optional['MerkleNode'] = None,right: Optional['MerkleNode'] = None):
        self.hash_value = hash_value
        self.left = left
        self.right = right
        
    # Creating leaf node
    @staticmethod
    def create_leaf_node(t: Transaction)->'MerkleNode':
        hash_input = (t.from_account + str(t.incentive) + t.to_account + str(t.amount)).encode('utf-8')
        #print("hash of " + t.from_account + "-" + str(t.incentive) + "-" + t.to_account + "-" + str(t.amount))
        return MerkleNode(calculate_hash(hash_input))
        
    # Creating non-leaf nodes
    @staticmethod
    def create_internal_node(l: 'MerkleNode', r: Optional['MerkleNode'])->'MerkleNode':
        if r:
            #print("Hash value of internal node with left " + l.hash_value + " and right " + r.hash_value)
            hash_input = (l.hash_value + r.hash_value).encode('utf-8')
            return MerkleNode(calculate_hash(hash_input),l,r)  
        else:
            #print("Hash value of internal node with left " + l.hash_value)
            return MerkleNode(l.hash_value)
        
          

### Build Function

In [57]:
def build_merkle_tree(transactions: List[Transaction])->Optional['MerkleNode']:
    # no transactions present in list
    if not transactions:
        return None
    '''
    create a list of leaf nodes using the transactions
    [ t1 t2 t3 t4 ... tn ]
    then create the tree
    '''
    nodes = [MerkleNode.create_leaf_node(t) for t in transactions]

    while(len(nodes)>1):
        new_nodes = []
        for i in range(0,len(nodes),2):
            # we can add left and right
            if i < len(nodes) - 1 :
                new_nodes.append(MerkleNode.create_internal_node(nodes[i],nodes[i+1]))
            # can only add one of them
            else:
                new_nodes.append(MerkleNode.create_internal_node(nodes[i],None))
        nodes = new_nodes

    return nodes[0]
            
                

    
    
    
        
    


    

### Function to create the entire merkle tree and return the root's hash value

In [58]:
def get_merkle_root(t: List[Transaction])->str:
    root = build_merkle_tree(t)
    return root.hash_value if root else "0"

## Blocks

#### Attributes

**block_number**: current block number

**merkle_root**: root of merkle_tree

**block_hash**: Hash(prev_block_hash+curr_block_number+current_merkle_root).(utf-8)

**prevBlockHash** = Hash(prev_block_hash+curr_block_number+current_merkle_root)

**transactions**: array of transactions (at most 3)

In [59]:
class Block:
    #Constructor
    def __init__(self,block_number: int,prev_block_hash: str,transaction_list: List[Transaction]):
        self.block_number = block_number
        self.prev_block_hash = prev_block_hash
        self.transaction_list = transaction_list
        self.merkle_root = get_merkle_root(transaction_list)
        self.block_hash = calculate_hash((prev_block_hash+str(block_number)+self.merkle_root).encode('utf-8'))  

### Execute Transactions

In [60]:
def execute_transaction(t: Transaction, balances: dict)->bool:
    balance_of_from = balances.get(t.from_account,0)
    balance_of_to = balances.get(t.to_account,0)
    # valid transaction only if balance of x exceeds amount being spent
    if balance_of_from >= t.amount:
        balances[t.from_account] = balances.get(t.from_account,0) - t.amount
        balances[t.to_account] = balances.get(t.to_account,0) + t.amount
        return True
    return False

### Prints Transaction Details

In [61]:
def get_transaction_details(transactions: List[Transaction]):
    all_ts = []
    for t in transactions:
        vals = [t.from_account,t.to_account,t.amount,t.incentive]
        all_ts.append(vals)
    print(all_ts)
        
     
    

# MAIN FUNCTION
    

In [ ]:
def main():

    blocks = []
    block_number = 1
    prev_block_hash = "0"
    current_transactions = []
    max_transactions_in_block = 4
    
    # get the account details of every individual
    n = int(input())
    balances = {}
    for i in range(n):
        account,balance = str(input()).split()
        balances[account] = int(balance)

    # get every transaction details
    number_of_transactions = int(input())
    transactions = []
    for i in range(number_of_transactions):
        from_account,to_account,amount,incentive = str(input()).split()
        transactions.append(Transaction(from_account,to_account,int(amount),int(incentive)))

    # sort based on decreasing order of incentive and increasing order of receiver (to_account)
    transactions = sorted(transactions,key = lambda t:(-t.incentive,t.to_account))
    
    # now create the blockchain
    for t in transactions:
        # if valid transaction
        if execute_transaction(t,balances):
            current_transactions.append(t)
            if len(current_transactions) == max_transactions_in_block:
                block = Block(block_number,prev_block_hash,current_transactions)
                prev_block_hash = block.block_hash
                current_transactions = []
                block_number = block_number + 1
                blocks.append(block)


 
    if len(current_transactions) != 0:
        block = Block(block_number,prev_block_hash,current_transactions)
        blocks.append(block)

    for b in blocks:
        print(b.block_number)
        print(b.block_hash)
        get_transaction_details(b.transaction_list)
        print(b.merkle_root)

        
        
            

    


if __name__ == "__main__":
    main()

        